#### Load Libraries

In [22]:
import pandas as pd

#### Load Data

In [46]:
user = pd.read_csv("data/_raw/users_data.csv")
gyms = pd.read_csv("data/_raw/gym_locations_data.csv")
checkins = pd.read_csv("data/_raw/checkin_checkout_history_updated.csv")
subscriptions = pd.read_csv("data/_raw/subscription_plans.csv")

In [47]:
# Looking at data structure
print(user.head())
print(gyms.head())
print(checkins.head())
print(subscriptions.head())

  user_id first_name last_name  age      gender   birthdate sign_up_date  \
0  user_1      Chris    Wilson   56      Female  2000-02-29   2023-02-06   
1  user_2    Michael    Miller   46  Non-binary  1978-12-22   2023-08-08   
2  user_3     Daniel     Smith   32      Female  1962-08-30   2021-01-11   
3  user_4      David     Smith   60        Male  2003-12-05   2023-08-07   
4  user_5      Chris     Jones   25      Female  2004-08-25   2021-01-08   

  user_location subscription_plan  
0        Denver             Basic  
1       Orlando               Pro  
2       Orlando             Basic  
3        Denver               Pro  
4        Denver             Basic  
  gym_id     location  gym_type  \
0  gym_1     New York   Premium   
1  gym_2  Los Angeles    Budget   
2  gym_3      Chicago    Budget   
3  gym_4      Houston   Premium   
4  gym_5      Phoenix  Standard   

                                       facilities  
0  Climbing Wall, Swimming Pool, Basketball Court  
1           

In [48]:
# Inspect column names and general structure
print(user.info())
print(gyms.info())
print(checkins.info())
print(subscriptions.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user_id            5000 non-null   object
 1   first_name         5000 non-null   object
 2   last_name          5000 non-null   object
 3   age                5000 non-null   int64 
 4   gender             5000 non-null   object
 5   birthdate          5000 non-null   object
 6   sign_up_date       5000 non-null   object
 7   user_location      5000 non-null   object
 8   subscription_plan  5000 non-null   object
dtypes: int64(1), object(8)
memory usage: 351.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gym_id      10 non-null     object
 1   location    10 non-null     object
 2   gym_type    10 non-null     object
 3   facilities  10

#### Clean Data

In [49]:
# Look for na values
print('User')
print(user.isna().sum())
print('\n')
print('Gyms')
print(gyms.isna().sum())
print('\n')
print('Checkins')
print(checkins.isna().sum())
print('\n')
print('Subscriptions')
print(subscriptions.isna().sum())

User
user_id              0
first_name           0
last_name            0
age                  0
gender               0
birthdate            0
sign_up_date         0
user_location        0
subscription_plan    0
dtype: int64


Gyms
gym_id        0
location      0
gym_type      0
facilities    0
dtype: int64


Checkins
user_id            0
gym_id             0
checkin_time       0
checkout_time      0
workout_type       0
calories_burned    0
dtype: int64


Subscriptions
subscription_plan    0
price_per_month      0
features             0
dtype: int64


In [50]:
# Remove duplicates
print("User Data Duplicates:", user.duplicated().sum())
print("Gyms Data Duplicates:", gyms.duplicated().sum())
print("Check-ins Data Duplicates:", checkins.duplicated().sum())
print("Subscriptions Data Duplicates:", subscriptions.duplicated().sum())

User Data Duplicates: 0
Gyms Data Duplicates: 0
Check-ins Data Duplicates: 0
Subscriptions Data Duplicates: 0


In [51]:
# Convert dates to datetime objects
user['birthdate'] = pd.to_datetime(user['birthdate'])
user['sign_up_date'] = pd.to_datetime(user['sign_up_date'])
checkins['checkin_time'] = pd.to_datetime(checkins['checkin_time']) 
checkins['checkout_time'] = pd.to_datetime(checkins['checkout_time']) 

# Convert categorical variables to category type
user['gender'] = user['gender'].astype('category')
gyms['gym_type'] = gyms['gym_type'].astype('category')
checkins['workout_type'] = checkins['workout_type'].astype('category')

user['user_location'] = user['user_location'].str.lower()
gyms['location'] = gyms['location'].str.lower()

In [52]:
# Merge dfs - MAYBE NOT DO THIS ANYWAS :|
user_checkins = pd.merge(checkins, user, on='user_id', how='left')
user_checkins = pd.merge(user_checkins, gyms, on='gym_id', how='left')
user_checkins = pd.merge(user_checkins, subscriptions, on='subscription_plan', how='left')
user_checkins['subscription_plan'] = user_checkins['subscription_plan'].astype('category')

user_checkins.head()

,user_id,gym_id,checkin_time,checkout_time,workout_type,calories_burned,first_name,last_name,age,gender,birthdate,sign_up_date,user_location,subscription_plan,location,gym_type,facilities,price_per_month,features
0,user_3291,gym_6,2023-09-10 15:55:00,2023-09-10 16:34:00,Weightlifting,462,Michael,Rodriguez,41,Female,1998-03-10,2023-04-08,atlanta,Pro,philadelphia,Budget,"Swimming Pool, Climbing Wall, Sauna",49.99,"Access to all facilities, Unlimited class acce..."
1,user_1944,gym_2,2023-04-13 20:07:00,2023-04-13 22:43:00,Yoga,1278,Michael,Garcia,24,Female,1985-11-28,2023-07-16,las vegas,Pro,los angeles,Budget,"Climbing Wall, Yoga Classes, Sauna",49.99,"Access to all facilities, Unlimited class acce..."
2,user_958,gym_7,2023-06-10 12:24:00,2023-06-10 13:49:00,Cardio,858,Emily,Rodriguez,37,Male,1986-03-12,2021-09-15,boston,Basic,san antonio,Premium,"Sauna, Basketball Court, Swimming Pool",19.99,"Access to basic gym facilities, Limited class ..."
3,user_811,gym_2,2023-05-23 17:11:00,2023-05-23 20:01:00,Yoga,1134,David,Miller,34,Female,1984-03-14,2023-04-25,las vegas,Student,los angeles,Budget,"Climbing Wall, Yoga Classes, Sauna",9.99,"Access to basic facilities, Limited class acce..."
4,user_4923,gym_10,2023-02-21 06:20:00,2023-02-21 08:02:00,Weightlifting,1049,Michael,Johnson,32,Female,1999-03-04,2022-12-29,austin,Pro,san jose,Premium,"Swimming Pool, Sauna, CrossFit",49.99,"Access to all facilities, Unlimited class acce..."


### Preprocessing Data

Emma - feature engineering for clustering

In [64]:
# Initialize df
user_features = pd.DataFrame()

# 1. Check-In Frequency
checkin_counts = checkins.groupby('user_id').size()
user_timespan = checkins.groupby('user_id')['checkin_time'].apply(lambda x: (x.max() - x.min()).days)
user_features['checkin_frequency'] = checkin_counts / (user_timespan / 30)

# 2. Time of Day Preference
checkins['hour'] = checkins['checkin_time'].dt.hour
bins = [0, 5, 12, 17, 21, 24]
labels = ['late night', 'morning', 'afternoon', 'evening', 'night']
checkins['time_of_day'] = pd.cut(checkins['hour'], bins=bins, labels=labels, right=False)
time_of_day_preference = checkins.groupby('user_id')['time_of_day'].agg(lambda x: x.mode()[0])
user_features['time_of_day_preference'] = time_of_day_preference

# 3. Workout type
workout_diversity = checkins.groupby('user_id')['workout_type'].nunique()
user_features['workout_diversity'] = workout_diversity

# 4. Seasonal Usage Pattern (std of monthly check-ins)
checkins['month'] = checkins['checkin_time'].dt.month
monthly_checkins = checkins.groupby(['user_id', 'month']).size().unstack(fill_value=0)
user_features['seasonal_variability'] = monthly_checkins.std(axis=1)

# 5. Weekend vs. Weekday Preference
checkins['is_weekend'] = checkins['checkin_time'].dt.dayofweek >= 5
weekend_preference = checkins.groupby('user_id')['is_weekend'].mean()
user_features['weekend_preference'] = weekend_preference

# 6. Average Workout Duration
checkins['workout_duration'] = (checkins['checkout_time'] - checkins['checkin_time']).dt.total_seconds() / 60
avg_workout_duration = checkins.groupby('user_id')['workout_duration'].mean()
user_features['avg_workout_duration'] = avg_workout_duration

# 7. Check-In Regularity (variance in days between check-ins)
checkins['days_since_last_checkin'] = checkins.groupby('user_id')['checkin_time'].diff().dt.days
regularity = checkins.groupby('user_id')['days_since_last_checkin'].std()
user_features['checkin_regularity'] = regularity

# Save to CSV
user_features.to_csv("data/user_features.csv", index=True)

In [65]:
user_features.head()

,checkin_frequency,time_of_day_preference,workout_diversity,seasonal_variability,weekend_preference,avg_workout_duration,checkin_regularity
user_id,,,,,,,
user_1,6.939502,morning,6,4.116363,0.276923,113.323077,112.540979
user_10,5.708955,morning,6,3.414023,0.313725,97.627451,123.878790
user_100,5.809859,afternoon,6,2.121320,0.345455,104.400000,121.367292
user_1000,4.907063,afternoon,6,2.503331,0.295455,100.977273,110.390869
user_1001,5.744681,morning,6,1.776388,0.259259,110.148148,114.652637


Joanna, Feature Creation for Clustering - JOANNA your old code. The variable names have just been changed for consistency

In [55]:
# Calculate workout length (in minutes) for each session and add as a column
user_checkins['workout_duration (min)'] = (
    user_checkins['checkout_time'] - user_checkins['checkin_time']
).dt.total_seconds() / 60

# Extract the hour of check-in and categorize into time-of-day groups
user_checkins['time_window'] = user_checkins['checkin_time'].dt.hour
user_checkins['time_of_day'] = pd.cut(
    user_checkins['time_window'],
    bins=[0, 5, 8, 11, 14, 17, 20, 23, 24],
    labels=['odd hours', 'early morning', 'morning', 'noon', 'afternoon', 'evening', 'late evening', 'odd hours'],
    include_lowest=True,
    ordered=False
)

# Remove the intermediate 'time_window' column, as it’s not needed further
user_checkins.drop(['time_window'], axis=1, inplace=True)

In [56]:
user_checkins.head()

,user_id,gym_id,checkin_time,checkout_time,workout_type,calories_burned,first_name,last_name,age,gender,...,sign_up_date,user_location,subscription_plan,location,gym_type,facilities,price_per_month,features,workout_duration (min),time_of_day
0,user_3291,gym_6,2023-09-10 15:55:00,2023-09-10 16:34:00,Weightlifting,462,Michael,Rodriguez,41,Female,...,2023-04-08,atlanta,Pro,philadelphia,Budget,"Swimming Pool, Climbing Wall, Sauna",49.99,"Access to all facilities, Unlimited class acce...",39.0,afternoon
1,user_1944,gym_2,2023-04-13 20:07:00,2023-04-13 22:43:00,Yoga,1278,Michael,Garcia,24,Female,...,2023-07-16,las vegas,Pro,los angeles,Budget,"Climbing Wall, Yoga Classes, Sauna",49.99,"Access to all facilities, Unlimited class acce...",156.0,evening
2,user_958,gym_7,2023-06-10 12:24:00,2023-06-10 13:49:00,Cardio,858,Emily,Rodriguez,37,Male,...,2021-09-15,boston,Basic,san antonio,Premium,"Sauna, Basketball Court, Swimming Pool",19.99,"Access to basic gym facilities, Limited class ...",85.0,noon
3,user_811,gym_2,2023-05-23 17:11:00,2023-05-23 20:01:00,Yoga,1134,David,Miller,34,Female,...,2023-04-25,las vegas,Student,los angeles,Budget,"Climbing Wall, Yoga Classes, Sauna",9.99,"Access to basic facilities, Limited class acce...",170.0,afternoon
4,user_4923,gym_10,2023-02-21 06:20:00,2023-02-21 08:02:00,Weightlifting,1049,Michael,Johnson,32,Female,...,2022-12-29,austin,Pro,san jose,Premium,"Swimming Pool, Sauna, CrossFit",49.99,"Access to all facilities, Unlimited class acce...",102.0,early morning


In [58]:
#Maybe not gather it to one df anyways

#user_checkins.to_csv("data/merged_data.csv", index=False)